In [1]:
import os
import numpy as np
from spectral import *
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn import preprocessing
import pickle

DATASTORE = 'C:\\Datasets\\bacterias'
spectral.settings.envi_support_nonlowercase_params = True


In [48]:
from sklearn.cluster import KMeans
from math import factorial
from scipy.signal import savgol_filter
import pickle

def get_fewer_lines(mat, ammount):
    n_mat = []
    r, _, _ = mat.shape
    for i in range(0, r, int(r/ammount)):
        n_mat.append(mat[i, :, :])
    return np.array(n_mat)

def calibration(I, W, D):
    row,column,wave = I.shape
    arr = np.copy(I)

    meanw = np.mean(W, axis=0)
    meand = np.mean(D, axis=0)

    for z in range(wave):
        if (z % 30 == 0):
            print('CAMADAS {}-{}'.format(z, 256 if z+30>256 else z+30))
        for x in range(row):
            for y in range(column):
                w = meanw[0,y,z]
                d = meand[0,y,z]
                s = I[x,y,z]

                den = w-d
                num = s-d
                if den and num/den > 0:
                    arr[x,y,z] = -math.log10(num / den)
                else:
                    arr[x,y,z] = 0
    return arr

def hsi2matrix(arr):
    if len(arr.shape) != 3:
        raise BaseException('A entrada deve possuir 3 dimensões')

    r, c, w = arr.shape
    return np.reshape(arr, (r*c, w))

def mat2hsi(mat, shape):
    return np.reshape(mat, shape)

def pca_95(x):
    scaled_data = preprocessing.scale(x)

    return PCA(n_components=0.95).fit_transform(scaled_data)

def get_clusters(x):
    pca_data = pca_95(x)
    km = KMeans(n_clusters=2).fit(pca_data)
    return km

def get_layer(hsi, layer):
    return hsi[:,:,layer]


def savitzky_golay_filter(y, window_size, order, deriv=0, rate=1):
    order_range = range(order+1)
    half_window = (window_size - 1) // 2
    b = np.mat([[k**i for i in order_range]
               for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    firstvals = y[0] - np.abs(y[1:half_window+1][::-1] - y[0])
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve(m[::-1], y, mode='valid')

def snv_filter(mat):
    nmat = np.copy(mat)
    mean = np.mean(mat, axis=1)
    std = np.std(mat, axis=1)
    for i in range(mat.shape[0]):
        nmat[i] = (nmat[i] - mean[i])/std[i]
    return nmat

def remove_pixels(mat, side, amount):
    cpy_mat = np.copy(mat)
    if side == 'top':
        cpy_mat[0:amount,:,:]=0
    elif side == 'left':
        cpy_mat[:,0:amount,:]=0
    elif side == 'right':
        cpy_mat[:,-amount:,:]=0
    else:
        cpy_mat[-amount:,:,:]=0
    return cpy_mat

def apply_mask(km,mat):
    mask1 = np.copy(mat)
    mask2 = np.copy(mat)
    lab = km.labels_
    for i in range(mat.shape[0]):
        if lab[i] == 0:
            mask1[i,:] = 0
        else:
            mask2[i,:] = 0
    
    return (mat2hsi(mask1, mat.shape) ,mat2hsi(mask2, mat.shape))


def hsi_remove_background(mat):
    mat_cpy = apply_filters(mat)
    km = get_clusters(mat_cpy)
    m1, m2 = apply_mask(km, mat)
    return (m1,m2)
    
def which_cluster_to_mantain(mask1, mask2):
    plt.figure()
    plt.title("FIGURE 1")
    plt.imshow(get_layer(mask1,10), cmap='gray')
    plt.figure()
    plt.title("FIGURE 2")
    plt.imshow(get_layer(mask2, 10), cmap='gray')
    plt.show()
    
    resp = int(input('Qual cluster deseja manter? (1/2)'))
    if resp != 1 and resp != 2:
        raise BaseException("Selected option not available.")
    
    return resp - 1
    
def get_hsi_data(path):
    orig_name = [a for a in os.listdir(path) if '.hdr' in a and 'DARK' not in a and 'WHITE' not in a]
    dark_name = [a for a in os.listdir(path) if '.hdr' in a and 'DARK' in a]
    white_name = [a for a in os.listdir(path) if '.hdr' in a and 'WHITE' in a]

    I = open_image(os.path.join(path, orig_name[0]))
    W = open_image(os.path.join(path, white_name[0]))
    D = open_image(os.path.join(path, dark_name[0]))

    return (I.load(), W.load(), D.load())

def get_no_background_pixels(mat):
    return np.where(mat != 0)

def apply_filters(mat):
    mat_cpy = np.copy(mat)
    for i in range(mat.shape[0]):
        mat_cpy[i] = savgol_filter(mat_cpy[i], 21, 2, 1)

    return snv_filter(mat_cpy)


def preprocess_training_data_full(choose_bac: int):
    """
        choose_bac is the bacteria to process (since takes forever to do all at once)
        returns a calibrated array based on dark and white hdr's, the pixels containing the bacteria (with no background) and the label for that bacteria
    """

    bac_dirs = os.listdir(DATASTORE)

    for ind, bac in enumerate(bac_dirs):
        if (choose_bac == ind):

            individual_bac_dir = os.path.join(DATASTORE, bac)

            I, W, D = get_hsi_data(individual_bac_dir)

            W = get_fewer_lines(W, 25)
            D = get_fewer_lines(D, 25)

            arr_calib = calibration(I, W, D)

            cube = preprocess_training_data_from_calibration(arr_calib)
            return [arr_calib, cube]
            
def preprocess_training_data_from_calibration(arr_calib):
    cube = replace_median(arr_calib)

    mat = hsi2matrix(cube)

    mask1, mask2 = hsi_remove_background(mat)
    mask1 = mat2hsi(mask1, arr_calib.shape)
    mask2 = mat2hsi(mask2, arr_calib.shape)

    cluster = which_cluster_to_mantain(mask1, mask2)
    retMat = mask1
    if cluster == 1:
        retMat = mask2

    return retMat[:, :, 1:256-14]

def replace_median(cube):
    x,y,z = cube.shape
    for i in range(z):
        rows, cols = np.where(cube[:,:,i] == 0)
        for j in range(len(rows)):
            if rows[j] > 1 and cols[j] > 1 and rows[j] < x - 1 and cols[j] < y - 1:
                wdn = cube[rows[j]-1:rows[j]+2, cols[j]-1: cols[j]+2, i]
                r, _ = np.where(wdn == 0)
                if len(r) == 1:
                    wdn = np.where(wdn != 0)
                    cube[rows[j], cols[j], i] = np.median(wdn)
    return cube

def save_all(path, calib, masked):
    pickle_out = open(os.path.join(path, 'calib.pickle'), "wb")
    pickle.dump(calib, pickle_out)

    pickle_out = open(os.path.join(path, 'masked.pickle'), "wb")
    pickle.dump(masked, pickle_out)

    pickle_out.close()

def load_calib(filename):
    pickle_in = open(filename, "rb")
    return pickle.load(pickle_in)

def plot_dif_with_ref_cube(mat, ref, noProcess):
    nn = hsi2matrix(mat)
    NN = hsi2matrix(ref)
    
    med1 = np.mean(nn, axis=0)
    med2 = np.mean(NN, axis=0)
    med3 = np.mean(noProcess, axis=0)

    print('RMSE: ', math.sqrt(np.mean(np.square(NN - nn))), 'Mean: ', np.mean(nn) - np.mean(NN))

    x = np.linspace(0, 241, 241)
    plt.plot(x, med1, '-g', label='minha')
    plt.plot(x, med2, '-b', label='ref')
    plt.legend()
    plt.figure()
    plt.plot(x, med3, '-r', label='sem proc')
    plt.legend()


def plot_imgs(img1, img2, wave):
    plt.imshow(img1[:,:,wave], cmap='gray')
    plt.figure()
    plt.imshow(img2[:,:,wave], cmap='gray')
    plt.show()


def get_dir_name(path, index):
    return os.listdir(path)[index]

def show_img_on_wave(cube, layer):
    mat = get_layer(cube, layer)
    plt.imshow(mat, cmap='gray')
    plt.show()

def print_dif_pixel(img1, img2):
    r,c,w = img1.shape
    out = 0
    for k in range(w):
        if (k % 30 == 0):
            print('CAMADAS {}-{}'.format(k, 256 if k+30>256 else k+30))
        for i in range(r):
            for j in range(c):
                if img1[i,j,k] - img2[i,j,k] > 0.1:
                    out = out + 1
    return out
